# 2章パフォーマンスの分析

## 2.1 パフォーマンス向上の方法

Pythonでは
* アルゴリズムの選択
* データ構造の選択
* 組み込み関数の利用
* Pythonのコンパイル
* 非同期のコード
* 並列・分散コンピューティング

## 2.2 実行時間の計測

数値リストの最頻値（mode）を計算する簡単なコードを例にして、コードの実行時間を測定する方法を見てみる

In [1]:
def slow_way_to_calculate_mode(list_of_numbers):  # 最頻値を計算する遅い方法
    result_dict = {}
    for i in list_of_numbers:
        if i not in result_dict:
            result_dict[i] = 1
        else:
            result_dict[i] += 1

    mode_vals = []
    max_frequency = max(result_dict.values())
    for key, value in result_dict.items():
        if value == max_frequency:
            mode_vals.append(key)

    return mode_vals

次に上の関数に入力する100,000個のランダムな整数のリストを作成する

In [2]:
import numpy as np


random_integers = np.random.randint(1, 1_000_000, 1_000_000)

関数の実行時間を測定する最も簡単な方法は、timeモジュールを用いること。  
実行環境の時計で、関数を実行する前と後の時刻を記録し、その差を出力する。

In [3]:
import time


start = time.time()
slow_way_to_calculate_mode(random_integers)
end = time.time()

print(f"{end - start}")

0.22549104690551758


ただし、関数の実行時間は変動する可能性があるので、複数回測定するのがよい。  
この場合、timeitモジュールが使える。

In [4]:
%%timeit
slow_way_to_calculate_mode(random_integers)

257 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


timeitモジュールの出力結果
例）`170 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)`  
- `170ms`：1回のループ（関数実行）にかかった平均時間 → **実行時間は平均170ミリ秒**だった。
- `± 12.1 ms`：平均時間に対する標準偏差（Standard Deviation） → **実行時間のバラつきが± 12.1ミリ秒**あった
- `of 7 runs`：timeitは内部で同じコードブロックの計測を何回か繰り返す。 → **7回計測**して、上の平均時間とその標準偏差を算出した
- `1 loop each`：各run の中で、そのコードを**1回だけ実行**した

最頻値を計算する別の方法と比較し、それによってパフォーマンスが向上するかどうかを見てみる。  
そこで最頻値を計算する別の方法を組む

In [5]:
from collections import Counter


def mode_using_counter(list_of_numbers):
    c = Counter(list_of_numbers)
    return c.most_common(1)[0][0]

In [6]:
%%timeit
mode_using_counter(random_integers)

211 ms ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


(実行のたびに多少時間が異なるので、具体的に言及しないが)  
`slow_way_to_calculate_mode()`と`mode_using_counter()`を比較すると、後者の方が関数の実行平均時間とその標準偏差がともに小さくなる。  
標準ライブラリなどの組み込み関数を用いると、自分で書くより速くなることが多い。

## 2.3 プロファイリング

2.2 で例示した関数より内容が長いものや、スクリプト全体を対象にしたい場合、行をセルに分割し、別々に計測するのは面倒。  
**プロファイラ**を用いると関数のどの部分で最も時間がかかっているかを教えてくれるほか、より詳しい情報も提供してくれるため、コードのボトルネックを発見しやすい。  
ここでは「メモリ使用量」をプロファイリングする方法に触れる。

### 2.3.1 cProfile

cProfileはPython組み込みのプロファイラ。  
ここでは乱数ジェネレーターを↑のmode関数の中に入れて、プロファイリングしてみる。

In [7]:
# 書籍ではimport節があるが、ここでは省略
def mode_using_counter(n_intergers):
    random_integers = np.random.randint(1, 100_000, n_intergers)
    c = Counter(random_integers)
    return c.most_common(1)[0][0]

In [8]:
%%prun
mode_using_counter(10_000_000)

         515 function calls (511 primitive calls) in 1.058 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.967    0.967    0.967    0.967 {built-in method _collections._count_elements}
        2    0.087    0.043    1.054    0.527 base_events.py:1962(_run_once)
        4    0.002    0.000    0.002    0.000 {built-in method builtins.max}
        2    0.001    0.000    0.001    0.000 {method '__exit__' of 'sqlite3.Connection' objects}
        1    0.001    0.001    1.056    1.056 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
       13    0.000    0.000    0.000    0.000 socket.py:623(send)
        2    0.000    0.000    0.967    0.484 {method 'control' of 'select.kqueue' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}

上の出力のうち`tottime`列を見ると、コードを実行したときに一番時間を費やした箇所は`{built-in method _collections._count_elements}`とあり、これはCounter関数である。

In [9]:
%load_ext snakeviz

In [10]:
%%snakeviz
mode_using_counter(10_000_000)

 
*** Profile stats marshalled to file '/var/folders/0v/_gfrv49n5g7_0jf5vdxbwx5h0000gn/T/tmpj7kduh7w'.
Embedding SnakeViz in this document...
<function display at 0x108f516c0>


もし↑で  
`*** Profile stats marshalled to file '/var/folders/0v/_gfrv49n5g7_0jf5vdxbwx5h0000gn/T/tmpj7kduh7w'.
Embedding SnakeViz in this document...
<function display at 0x108f516c0>
`  
と出力したら、下のコマンドをVSCode内のターミナルで実行することで標準設定しているブラウザにて表示できる。cProfileの出力結果は作業環境のVSCodeとの相性が悪いようだ。  
`snakeviz <path>`  
※pathに`*** Profile stats marshalled to file`以降に出力されたパスを入れる。  
(この場合は/var/folders/0v/_gfrv49n5g7_0jf5vdxbwx5h0000gn/T/tmpj7kduh7wであるので、`snakeviz /var/folders/0v/_gfrv49n5g7_0jf5vdxbwx5h0000gn/T/tmpj7kduh7w`)

### 2.3.2 line_profiler

cProfileは、Pythonの内部動作の詳細を含め、処理にてどのように時間を費やしているのか詳細な内訳を教えてくれる一方  
コードから呼び出されるPythonの組み込み関数に深入りしているがゆえに、cProfileで得られる出力が分かりづらい。  
そこでline_profilerをを用いることでよりわかりやすくすることができる。

In [11]:
%load_ext line_profiler

In [12]:
%lprun -f mode_using_counter mode_using_counter(10_000_000)

Timer unit: 1e-09 s

Total time: 0.987013 s
File: /var/folders/0v/_gfrv49n5g7_0jf5vdxbwx5h0000gn/T/ipykernel_64547/1731041392.py
Function: mode_using_counter at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def mode_using_counter(n_intergers):
     3         1  110920000.0 1.11e+08     11.2      random_integers = np.random.randint(1, 100_000, n_intergers)
     4         1  874567000.0 8.75e+08     88.6      c = Counter(random_integers)
     5         1    1526000.0 1.53e+06      0.2      return c.most_common(1)[0][0]

line_profilerの出力結果の`% Time`でその関数を実行、処理した時間に対する該当行の割合を確認できる。

### 2.3.3 Memrayによるメモリプロファイリング

In [14]:
!memray run mode_using_counter.py

Writing profile results into memray-mode_using_counter.py.77444.bin
28057
[memray] Successfully generated profile results.

You can now generate reports from the stored allocation records.
Some example commands to generate reports:

/Users/Shared/Learning/Books/books_Software_Engineering_for_Data_Scientists/.venv/bin/python3 -m memray flamegraph memray-mode_using_counter.py.77444.bin


In [ ]:
# !memray flamegraph memray-mode_using_counter.py.77444.bin

  Calculating high watermark... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00--:--
  Processing allocation records... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00--:--
Wrote memray-flamegraph-mode_using_counter.py.77444.html


## 2.4 時間計算量

### 2.4.1 時間計算量の見積もり

In [16]:
def weighted_mean(list_of_numbers, weights):
    running_total = 0
    for i in range(len(list_of_numbers)):
        running_total += (list_of_numbers[i] * weights[i])
    return (running_total / sum(weights))

In [17]:
def covariance(X, Y):
    cov_sum = 0
    for i in range(len(X)):
        for j in range(len(Y)):
            cov_sum += 0.5 * (X[i] - X[j]) * (Y[i] - Y[j])
    return cov_sum / (len(X) ** 2)

In [18]:
def covariance_fast(X, Y):
    avg_X = sum(X) / len(X)
    avg_Y = sum(Y) / len(Y)

    result = 0
    for i in range(len(X)):
        result += (X[i] - avg_X) * (Y[i] - avg_Y)

    return result / len(X)

### 2.4.2 オーダー記法